In [19]:
import os
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import scipy as sp
import sensuous.preprocessing.csv as prep
from sensuous.model.knn import strip_artists, predict_playlist_csv
from sklearn.preprocessing import MinMaxScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
prep.preprocess_csv_data()

Csv data preprocessing done, saved the cleaned data to file "../data/all-songs.csv"


In [21]:
df = pd.read_csv('../data/all-songs.csv')

In [22]:
df = df.drop(columns='Unnamed: 0')

In [23]:
df.iloc[150500]

artists         ['VELVETEARS']
song_title                None
danceability             0.694
energy                   0.407
key                          5
liveness                0.0785
loudness               -14.235
speechiness             0.0332
tempo                   90.023
valence                  0.151
Name: 150500, dtype: object

In [11]:
song_title = 'I Miss You'
artist = 'Teddy Pendergrass'

In [15]:
predict_playlist_csv(artist, song_title, data_dir='../data')

There is more than one entry. Returning the user's entry.


Unnamed: 0                         0.000004
artists                      ['Elton John']
song_title      Sixty Years On - Piano Demo
danceability                       0.457831
energy                                0.368
key                                0.545455
liveness                              0.156
loudness                           0.764086
speechiness                        0.028986
tempo                              0.575171
valence                               0.259
Name: 1, dtype: object

In [12]:
[(artist in df['artists']) | (('feat. ' + artist) in df['song_title'])]

[False]

In [15]:
df.dropna().shape

(226022, 10)

In [18]:
type(df.iloc[150500].song_title)

str